# Abdullah Alhussni - aa10108
## Applied Machine Learning - ENGR-UH 3332 - Project 3

In [1]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt

iris_df = pd.read_csv("iris.csv", names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
with pd.option_context('future.no_silent_downcasting', True):
    iris_df = iris_df.replace({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}).infer_objects()

spam_df = pd.read_csv("spambase.csv")
spam_df.columns = list(spam_df.columns[:-1]) + ["class"] # adding the "class" is necessary for our node definition

In [2]:
class node:
    def __init__(self, data, n_min):
        self.columns = data.columns
        self.n_classes = data["class"].nunique()
        self.list_of_classes = data["class"].unique()
        data = data.to_numpy()
        self.features = data[:, :-1]
        self.target = data[:, -1]
        self.n_instances = self.features.shape[0]
        self.n_features = self.features.shape[1]
        self.n_min = n_min
        self.min_per_leaf = self.n_instances * n_min // 100
        self.branchr = None
        self.branchl = None
        self.split_feature = None
        self.split_condition = None
        freq = np.zeros(self.n_classes)
        for i in range(self.n_classes):
            condition = self.target == self.list_of_classes[i]
            freq[i] = len(self.target[condition])
        self.label = self.list_of_classes[np.argmax(freq)]

    def putback(self, features, target): # puts any split/branch into a dataframe again so it can be reconstructed into a node
        data = pd.DataFrame(features, columns = self.columns[:-1])
        data['class'] = target
        n_min = self.n_min
        a = node(data, n_min)
        a.min_per_leaf = self.min_per_leaf # makes sure the min_per_leaf doesn't change when we split due to n_instances changing,
        # then min_per_leaf is determined by the root training node
        return a

    def condition_split(self, split_feature, split_condition):
        if split_feature != None or split_condition != None:
            if self.features.dtype == 'bool':
                condition = self.features == True
            else:
                condition = self.features[:, split_feature] >= split_condition

            branch1features = self.features[np.where(condition)[0]]
            branch2features = self.features[np.where(~condition)[0]]
            branch1target = self.target[np.where(condition)[0]]
            branch2target = self.target[np.where(~condition)[0]]

            if len(branch1target) != 0 and len(branch2target) != 0:
                branchr = self.putback(branch1features, branch1target)
                branchl = self.putback(branch2features, branch2target)
                self.branchr = branchr
                self.branchl = branchl
                self.branchr.min_per_leaf = self.min_per_leaf
                self.branchl.min_per_leaf = self.min_per_leaf

    def optimal_split(self):
        IG_max = 0
        best_feature = None
        best_split_value = None
        if self.n_classes > 1 and self.n_instances > self.min_per_leaf:
            for i in range(self.n_features):
                if self.features[:, i].dtype == 'bool':
                    IG = self.IG(i, True)
                    self.condition_split(i, True)
                    if self.branchr == None or self.branchl == None:
                        return None, None # the recursive_split function tests the output of optimal_split and if they're both
                        # self, we understand that the optimal split is no split, so just return self in recursive_split
                    if IG > IG_max:
                        IG_max = IG
                        best_feature = i
                        best_split_value = True
                else:
                    a = np.unique(self.features[:, i])
                    for j in range(len(a) - 1):
                        b = (a[j] + a[j + 1]) / 2
                        IG = self.IG(i, b)
                        self.condition_split(i, b)
                        if self.branchr == None or self.branchl == None:
                            continue
                        if IG > IG_max:
                            IG_max = IG
                            best_feature = i
                            best_split_value = b
            return best_feature, best_split_value
        else:
            return None, None # same as above

    def recursive_split(self):
        best_split_feature, best_split_condition = self.optimal_split()
        self.split_feature = best_split_feature
        self.split_condition = best_split_condition
        self.condition_split(best_split_feature, best_split_condition)
        if self.branchr != None and self.branchl != None:
            self.branchl.recursive_split(), self.branchr.recursive_split()
        elif self.branchr != None:
            self.branchr.recursive_split()
        elif self.branchl != None:
            self.branchl.recursive_split()

    def entropy(self):
        probs = np.zeros(self.n_classes)
        for i in range(self.n_classes):
            condition = self.target == self.list_of_classes[i]
            probs[i] = len(self.target[condition])
        probs = probs / self.n_instances
        probs = np.where(probs == 0, 1, probs) # replacing any probability 0 with 1 is to avoid log2(0)
        # which returns the same term, 0
        entropy = np.sum(-np.dot(probs, np.log2(probs)))
        return entropy

    def IG(self, split_feature, split_condition):
        self.condition_split(split_feature, split_condition)
        if self.branchr == None or self.branchl == None:
            return 0
        else:
            return self.entropy() - (self.branchr.n_instances * self.branchr.entropy() + self.branchl.n_instances * self.branchl.entropy()) / (self.n_instances)

    def create_decision_tree(self):
        self.recursive_split()
        print_binary_tree(self)

In [3]:
def print_binary_tree(root, indent = "", last = 'updown'):
    if root is not None:
        nb_spaces = "   "
        if last == 'up':
            if root.split_feature != None and root.split_condition != None:
                print(f"{indent}┌──Label: {root.label}, Feature_{root.split_feature} >= {root.split_condition}")
                indent += nb_spaces
            else:
                print(f"{indent}┌──Label: {root.label}")
                indent += nb_spaces
        elif last == 'down':
            if root.split_feature != None and root.split_condition != None:
                print(f"{indent}└──Label: {root.label}, Feature_{root.split_feature} >= {root.split_condition}")
                indent += nb_spaces
            else:
                print(f"{indent}└──Label: {root.label}")
                indent += nb_spaces
        elif last == 'updown':
            if root.split_feature != None and root.split_condition != None:
                print(f"{indent}Label: {root.label}, Feature_{root.split_feature} >= {root.split_condition}")
                indent += nb_spaces
            else:
                print(f"{indent}Label: {root.label}")
                indent += nb_spaces

        print_binary_tree(root.branchl, indent, 'up')
        print_binary_tree(root.branchr, indent, 'down')

In [4]:
def predict(node, point): # the argument is node but technically it's the tree, but you should call recursive_split() first
    position = node
    while True:
        if position.branchr == None or position.branchl == None:
            return position.label
        else:
            i = position.split_feature
            j = position.split_condition
            if j == True:
                position = position.branchr
            elif j == False:
                position = position.branchl
            elif point[i] >= j:
                position = position.branchr
            else:
                position = position.branchl

def accuracy_test(node, test):
    X_test = test.features
    y_test = test.target
    df = np.column_stack((X_test, y_test))
    num = test.n_instances
    y_pred = []
    for i in range(num):
        y_pred.append(predict(node, df[i, :]))

    return accuracy_score(y_test, y_pred)

In [5]:
n_min_iris = [5, 10, 15, 20]
n_min_spam = [5, 10, 15, 20, 25]

iris = node(iris_df, n_min_iris[1])

print("Iris Dataset Decision Tree")
iris.create_decision_tree()

spam = node(spam_df, n_min_spam[1])

print("Spam Dataset Decision Tree")
spam.create_decision_tree()

Iris Dataset Decision Tree
Label: 0, Feature_2 >= 2.45
   ┌──Label: 0.0
   └──Label: 1.0, Feature_3 >= 1.75
      ┌──Label: 1.0, Feature_2 >= 4.95
         ┌──Label: 1.0, Feature_3 >= 1.65
            ┌──Label: 1.0
            └──Label: 2.0
         └──Label: 2.0
      └──Label: 2.0, Feature_2 >= 4.85
         ┌──Label: 2.0
         └──Label: 2.0
Spam Dataset Decision Tree
Label: 0, Feature_52 >= 0.0555
   ┌──Label: 0.0, Feature_6 >= 0.055
      ┌──Label: 0.0, Feature_51 >= 0.191
         ┌──Label: 0.0, Feature_24 >= 0.025
            ┌──Label: 0.0, Feature_55 >= 10.5
               ┌──Label: 0.0, Feature_15 >= 0.845
                  ┌──Label: 0.0, Feature_7 >= 0.035
                     ┌──Label: 0.0, Feature_26 >= 0.125
                        ┌──Label: 0.0, Feature_18 >= 0.935
                           ┌──Label: 0.0
                           └──Label: 0.0
                        └──Label: 0.0
                     └──Label: 0.0
                  └──Label: 0.0
               └──Lab

In [ ]:
k = 10    
seed = 42
np.random.seed(seed)
kf = KFold(n_splits = k, shuffle = True, random_state = seed)

def decision_tree_accuracy(df, n_min):
    data = node(df, n_min)
    df = np.column_stack((data.features, data.target))
    accuracies = []
    for train, test in kf.split(df):
        X_train = df[train, :-1]
        y_train = df[train, -1]
        X_test = df[test, :-1]
        y_test = df[test, -1]

        data_train = data.putback(X_train, y_train)
        data_test = data.putback(X_test, y_test)

        data_train.recursive_split()
        accuracy = accuracy_test(data_train, data_test)
        accuracies.append(accuracy)
        mean_accuracy = np.mean(accuracies)
        stdev = np.std(accuracies)
    return mean_accuracy, stdev

def decision_tree_table(df, n_min):
    accuracies = []
    stdevs = []
    
    for n in n_min:
        x, y = decision_tree_accuracy(df, n)
        accuracies.append(x)
        stdevs.append(y)

    # Convert mean accuracies to percentages
    accuracy_percent = [f"{accuracy * 100:.2f}%" for accuracy in accuracies]
    
    # Convert stdevs to percentage of the mean accuracy
    stdev_percent = [f"{(stdev / accuracies[i]) * 100:.2f}%" for i, stdev in enumerate(stdevs)]

    results_df = pd.DataFrame({
        'n_min': n_min,
        'Mean Accuracy': accuracy_percent,
        'Standard Deviation': stdev_percent
    })
    
    print(results_df)

print("Iris Dataset Decision Tree Accuracy for Different Values of n_min")
decision_tree_table(iris_df, n_min_iris)
print("Spam Dataset Decision Tree Accuracy for Different Values of n_min")
decision_tree_table(spam_df, n_min_spam)

Iris Dataset Decision Tree Accuracy for Different Values of n_min
   n_min Mean Accuracy Standard Deviation
0      5        94.67%              5.27%
1     10        94.67%              5.27%
2     15        94.67%              5.27%
3     20        94.67%              5.27%
Spam Dataset Decision Tree Accuracy for Different Values of n_min
